In [15]:
# ==============================================================================
# CELL 1: CÀI ĐẶT, IMPORTS & THIẾT LẬP BAN ĐẦU
# ==============================================================================
import os
import json
import asyncio
import operator
from typing import TypedDict, Annotated, List, Dict, Any, Literal, Optional, Union

# Tải các biến môi trường
from dotenv import load_dotenv 
load_dotenv()

# LangChain & LangGraph
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# Thư viện hỗ trợ & Kho tri thức
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from rich.console import Console
from rich.markdown import Markdown
from rich.pretty import pprint
from IPython.display import Image, display

# --- CẤU HÌNH LOG ---
VERBOSE_MODE = True

# --- KHỞI TẠO CÁC CÔNG CỤ IN ẤN ---
console = Console()

def print_step(message: str):
    if VERBOSE_MODE:
        console.print(f"\n[bold cyan]>[/bold cyan] {message}")

def print_result(data: Any, title: str = "Kết quả"):
    if VERBOSE_MODE:
        console.print(f"[bold green]✔️ {title}:[/bold green]")
        pprint(data, expand_all=True)
        
def print_warning(message: str):
    if VERBOSE_MODE:
        console.print(f"[bold yellow]⚠️  {message}[/bold yellow]")

# --- KHỞI TẠO LLM ---
try:
    llm = ChatGroq(
        temperature=0.1, model="llama3-70b-8192",
        api_key=os.getenv("GROQ_API_KEY"), max_tokens=4096,
        max_retries=2
    )
    print("✅ LLM (Groq) đã được khởi tạo thành công.")
except Exception as e:
    print(f"❌ LỖI: Không thể khởi tạo LLM. Lỗi: {e}")
    llm = None

# --- TẢI KHO TRI THỨC ---
VECTOR_STORE_PATH = "vector_store/sgk_toan_9"
vector_store = None
if os.path.exists(VECTOR_STORE_PATH):
    try:
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        vector_store = FAISS.load_local(VECTOR_STORE_PATH, embeddings, allow_dangerous_deserialization=True)
        print("✅ Kho tri thức RAG đã được tải thành công.")
    except Exception as e:
        print(f"❌ LỖI khi tải kho tri thức: {e}")
else:
    print(f"❌ LỖI: Không tìm thấy kho tri thức tại '{VECTOR_STORE_PATH}'.")

✅ LLM (Groq) đã được khởi tạo thành công.
✅ Kho tri thức RAG đã được tải thành công.


In [16]:
# ==============================================================================
# CELL 2: KIẾN TRÚC DỮ LIỆU (STATE & PYDANTIC MODELS)
# ==============================================================================
from typing import TypedDict, Annotated, List, Dict, Any, Literal, Optional
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langchain_core.pydantic_v1 import BaseModel, Field

# --- ĐỊNH NGHĨA STATE ---
def merge_dicts(dict1: Dict[str, Any], dict2: Dict[str, Any]) -> Dict[str, Any]:
    merged = dict1.copy()
    for key, value in dict2.items():
        if key in merged and isinstance(merged[key], dict) and isinstance(value, dict):
            merged[key] = merge_dicts(merged[key], value)
        else:
            merged[key] = value
    return merged

class DetailedTask(TypedDict):
    task_id: int
    task_name: str
    task_description: str
    estimated_duration: int 
    status: Literal["pending", "completed"]

class TeacherStateV4(TypedDict):
    original_request: str
    messages: Annotated[List[BaseMessage], add_messages]
    analyzed_objective: Optional[Any]
    pedagogy_strategy: Optional[Any]
    expanded_queries: Optional[List[str]]
    task_list: Optional[List[DetailedTask]]
    current_task_id: Optional[int]
    reflection_notes: Optional[str]
    agent_outputs: Annotated[Dict[str, Any], merge_dicts] 
    final_lesson_plan: Optional[str]
    next_agent: str
    
    # Các trường mới cho phiên bản tự sửa lỗi
    current_content_to_validate: Optional[Dict[str, Any]]
    validation_feedback: Optional[str]
    domain: Optional[str]
    student_persona: Optional[Dict[str, Any]]
    pedagogical_blueprint: Optional[List[str]]


print("✅ Cấu trúc State 'TeacherStateV4' đã được định nghĩa.")

# --- ĐỊNH NGHĨA CÁC PYDANTIC MODELS ---
class LearningActivity(BaseModel):
    activity_name: str = Field(description="Tên của hoạt động học tập này thật hấp dẫn và rõ ràng.")
    description: str = Field(description="Mô tả chi tiết các bước giáo viên và học sinh cần thực hiện trong hoạt động.")
    duration_minutes: int = Field(description="Thời gian ước tính (bằng SỐ PHÚT) để hoàn thành hoạt động.")
    activity_type: str = Field(description="Phân loại hoạt động (ví dụ: 'Giảng giải lý thuyết', 'Luyện tập cặp đôi', 'Thảo luận nhóm', 'Trò chơi').")
    solution_guide: Optional[Any] = Field(description="Hướng dẫn giải, đáp án chi tiết hoặc các điểm chính cần lưu ý.")

class AssessmentItem(BaseModel):
    question: str = Field(description="Nội dung câu hỏi hoặc đề bài toán, được viết rõ ràng, dễ hiểu.")
    question_type: str = Field(description="Loại câu hỏi (ví dụ: 'Trắc nghiệm', 'Tự luận', 'Chứng minh').")
    options: Optional[List[str]] = Field(description="Các lựa chọn nếu là câu hỏi trắc nghiệm.")
    answer: str = Field(description="Đáp án chính xác và ngắn gọn cho câu hỏi.")
    solution_guide: Any = Field(description="Hướng dẫn giải chi tiết từng bước để học sinh có thể tự kiểm tra.")

class DetailedTaskModel(BaseModel):
    task_name: str = Field(description="Tên của nhiệm vụ này.")
    task_description: str = Field(description="Mô tả chi tiết nhiệm vụ cần thực hiện.")
    estimated_duration: int = Field(description="Thời gian ước tính (bằng phút) cho nhiệm vụ này.")

class TaskListWithDuration(BaseModel):
    tasks: List[DetailedTaskModel]

class ParsedObjective(BaseModel):
    action_verb: str
    bloom_level: int
    topic: str
    grade_level: str
    duration_minutes: Optional[int]

class PedagogyChoice(BaseModel):
    chosen_pedagogy: str
    pedagogy_rationale: str

class ExpandedQueries(BaseModel):
    queries: List[str]

class BestSnippets(BaseModel):
    best_snippets: List[str]

class TaskClassification(BaseModel):
    agent_category: Literal["activity_designer", "theory_synthesizer", "assessment_creator"]
    
class ValidationResult(BaseModel):
    is_valid: bool
    feedback: str
    
class Domain(BaseModel):
    domain: str
    
class StudentPersona(BaseModel):
    learning_pace: Literal["nhanh", "trung bình", "chậm"]
    engagement_style: Literal["chủ động", "thụ động", "hỗn hợp"]
    special_notes: str
    
class PedagogicalBlueprint(BaseModel):
    blueprint: List[str]

print("✅ Tất cả các Pydantic Models đã được định nghĩa.")

✅ Cấu trúc State 'TeacherStateV4' đã được định nghĩa.
✅ Tất cả các Pydantic Models đã được định nghĩa.


In [17]:
# ==============================================================================
# CELL 3: AGENTS - GIAI ĐOẠN PHÂN TÍCH & LẬP CHIẾN LƯỢC
# ==============================================================================

SYSTEM_PERSONA_PROMPT = "BẠN LÀ MỘT TRỢ LÝ AI CHUYÊN NGHIỆP, ĐÓNG VAI TRÒ MỘT GIÁO VIÊN GIÀU KINH NGHIỆM TẠI VIỆT NAM. LUÔN LUÔN trả lời bằng TIẾNG VIỆT."

async def objective_interpreter_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Objective Interpreter` đang phân tích mục tiêu...")
    prompt = f"{SYSTEM_PERSONA_PROMPT}\n**NHIỆM VỤ:** Đọc yêu cầu của người dùng và trích xuất các thông tin sau.\n**YÊU CẦU:** \"{state['original_request']}\"\n**CÁC TRƯỜNG CẦN TRÍCH XUẤT:** `action_verb`, `bloom_level` (SỐ NGUYÊN), `topic`, `grade_level`, `duration_minutes`.\n**CHỈ TRẢ VỀ JSON.**"
    structured_llm = llm.with_structured_output(ParsedObjective, method="json_mode")
    try:
        parsed_result = await structured_llm.ainvoke(prompt)
        analyzed_objective_dict = parsed_result.dict()
        analyzed_objective_dict['constraints'] = {'duration_minutes': parsed_result.duration_minutes}
        del analyzed_objective_dict['duration_minutes']
        print_result(analyzed_objective_dict, "Mục tiêu & Ràng buộc đã phân tích")
        return {"analyzed_objective": analyzed_objective_dict}
    except Exception as e:
        print_warning(f"Lỗi tại Objective Interpreter: {e}. Sử dụng mục tiêu mặc định.")
        return {"analyzed_objective": { "action_verb": "soạn", "bloom_level": 3, "topic": state['original_request'], "grade_level": "9", "constraints": {"duration_minutes": 90} }}

async def pedagogy_strategist_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Pedagogy Strategist` đang chọn phương pháp sư phạm...")
    prompt = f"{SYSTEM_PERSONA_PROMPT}\n**NHIỆM VỤ:** Dựa trên mục tiêu bài học, chọn MỘT phương pháp sư phạm và giải thích ngắn gọn.\n**MỤC TIÊU:** {state.get('analyzed_objective')}\n**YÊU CẦU ĐỊNH DẠNG:** Trả về một đối tượng JSON với 2 keys: `chosen_pedagogy` và `pedagogy_rationale`."
    structured_llm = llm.with_structured_output(PedagogyChoice, method="json_mode")
    try:
        response = await structured_llm.ainvoke(prompt)
        print_result(response.dict(), "Chiến lược sư phạm đã chọn")
        return {"pedagogy_strategy": response.dict()}
    except Exception as e:
        print_warning(f"Lỗi tại Pedagogy Strategist: {e}. Sử dụng chiến lược mặc định.")
        return {"pedagogy_strategy": {"chosen_pedagogy": "Dạy học giải quyết vấn đề", "pedagogy_rationale": "Mặc định do lỗi."}}

In [18]:
# ==============================================================================
# CELL 4: AGENTS - GIAI ĐOẠN THU THẬP & LẬP KẾ HOẠCH CHI TIẾT
# ==============================================================================

def update_agent_outputs(state: "TeacherStateV4", key: str, value: Any) -> Dict[str, Any]:
    outputs = state.get("agent_outputs", {}).copy()
    if key not in outputs: outputs[key] = []
    outputs[key].append(value)
    return {"agent_outputs": outputs}

async def query_expansion_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Query Expansion` đang mở rộng truy vấn tìm kiếm...")
    objective = state.get('analyzed_objective', {})
    prompt = f"{SYSTEM_PERSONA_PROMPT}\n**NHIỆM VỤ:** Tạo ra các cụm từ tìm kiếm đa dạng bằng tiếng Việt để tìm tài liệu.\n**CHỦ ĐỀ:** \"{objective.get('topic', '')}\"\n**YÊU CẦU ĐỊNH DẠNG:** Trả về JSON với key `queries`."
    structured_llm = llm.with_structured_output(ExpandedQueries, method="json_mode")
    try:
        response = await structured_llm.ainvoke(prompt)
        print_result(response.queries, "Các truy vấn tìm kiếm đã được mở rộng")
        return {"expanded_queries": response.queries}
    except Exception as e:
        print_warning(f"Lỗi tại Query Expansion: {e}. Sử dụng truy vấn gốc.")
        return {"expanded_queries": [objective.get('topic', '')]}

async def resource_scout_agent_v2(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Resource Scout` đang tìm kiếm và sàng lọc tài liệu...")
    retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    all_docs = []
    queries = state.get('expanded_queries', [])
    for query in queries:
        all_docs.extend(retriever.invoke(query))
    unique_docs_content = list({doc.page_content for doc in all_docs})[:8]
    print_step(f"Tìm thấy {len(all_docs)} tài liệu, sàng lọc còn {len(unique_docs_content)} đoạn văn bản độc nhất.")
    
    rerank_prompt = f"{SYSTEM_PERSONA_PROMPT}\n**NHIỆM VỤ:** Đọc yêu cầu gốc và chọn ra 3-4 đoạn văn bản tiếng Việt phù hợp NHẤT từ danh sách dưới đây.\n**YÊU CẦU GỐC:** \"{state.get('original_request', '')}\"\n**DANH SÁCH TÀI LIỆU:** {json.dumps(unique_docs_content, ensure_ascii=False)}\n**YÊU CẦU ĐỊNH DẠNG:** Trả về JSON với key `best_snippets`."
    structured_llm_reranker = llm.with_structured_output(BestSnippets, method="json_mode")
    best_snippets_text = "\n\n---\n\n".join(unique_docs_content)
    try:
        reranked_result = await structured_llm_reranker.ainvoke(rerank_prompt)
        best_snippets_text = "\n\n---\n\n".join(reranked_result.best_snippets)
    except Exception as e:
        print_warning(f"Lỗi tại Re-ranker: {e}. Sử dụng tất cả tài liệu.")

    summary_prompt = f"{SYSTEM_PERSONA_PROMPT}\n**NHIỆM VỤ:** Dựa vào các đoạn văn bản sau, tóm tắt các kiến thức cốt lõi nhất về chủ đề \"{state.get('analyzed_objective', {}).get('topic', '')}\".\n**QUY TẮC:** Tóm tắt phải cô đọng, mạch lạc, tập trung vào định nghĩa và định lý chính.\n**CÁC ĐOẠN VĂN BẢN:**\n{best_snippets_text}"
    summary = "Chưa có tóm tắt."
    try:
        summary_response = await llm.ainvoke(summary_prompt)
        summary = summary_response.content
    except Exception as e:
        print_warning(f"Lỗi tại bước tóm tắt: {e}.")
        
    resource = {"source": "Sách giáo khoa (từ RAG - đã sàng lọc)", "summary": summary}
    print_result(resource, "Tài liệu RAG cuối cùng")
    return update_agent_outputs(state, "resources", resource)

async def plan_delegator_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Plan Delegator` đang lập kế hoạch chi tiết...")
    critic_feedback_prompt = f"**PHẢN HỒI TỪ LẦN TRƯỚC (CẦN SỬA):** \"{state.get('reflection_notes', '')}\"."
    prompt = f"""{SYSTEM_PERSONA_PROMPT}
    **NHIỆM VỤ:** Xây dựng một kế hoạch bài dạy chi tiết (danh sách các nhiệm vụ).
    **MỤC TIÊU BÀI HỌC:** {state.get('analyzed_objective')}
    **KIẾN THỨC NỀN TẢNG:** {state.get('agent_outputs', {}).get('resources', [{}])[0].get('summary', 'Chưa có tóm tắt.')}
    {critic_feedback_prompt}
    **QUY TẮC:**
    1.  **Cấu trúc Logic:** Bắt đầu bằng giới thiệu, tiếp theo là hình thành kiến thức, sau đó là luyện tập, và cuối cùng là đánh giá/tổng kết.
    2.  **Thời gian Chu toàn:** Phân bổ thời gian cho các nhiệm vụ sao cho tổng thời gian gần bằng {state.get('analyzed_objective', {}).get('constraints', {}).get('duration_minutes', 90)} phút.
    **YÊU CẦU ĐỊNH DẠNG:** Trả về JSON với key `tasks`, mỗi task có `task_name`, `task_description`, `estimated_duration`.
    """
    structured_llm = llm.with_structured_output(TaskListWithDuration, method="json_mode")
    try:
        task_list_result = await structured_llm.ainvoke(prompt)
        tasks_with_status = [{"task_id": i, **t.dict(), "status": "pending"} for i, t in enumerate(task_list_result.tasks) if t]
        print_result(tasks_with_status, f"Đã tạo {len(tasks_with_status)} nhiệm vụ chi tiết")
        new_agent_outputs = {"resources": state.get("agent_outputs", {}).get("resources", [])}
        return {"task_list": tasks_with_status, "reflection_notes": None, "agent_outputs": new_agent_outputs}
    except Exception as e:
        print_warning(f"Lỗi tại Plan Delegator: {e}. Tạo task mặc định.")
        return {"task_list": [{"task_id": 0, "task_name": "Hoạt động luyện tập", "task_description": "Học sinh thực hành bài tập về đường tròn ngoại tiếp.", "estimated_duration": 45, "status": "pending"}], "reflection_notes": None, "agent_outputs": state.get("agent_outputs", {})}

In [19]:
# ==============================================================================
# CELL 5: AGENTS - GIAI ĐOẠN THỰC THI
# ==============================================================================

async def theory_synthesizer_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Theory Synthesizer` đang soạn nội dung lý thuyết...")
    task_to_run = next((t for t in state.get('task_list', []) if t['task_id'] == state.get('current_task_id')), None)
    if not task_to_run: return {"current_content_to_validate": {"error": "Không tìm thấy task."}}
    
    prompt = f"""{SYSTEM_PERSONA_PROMPT}
    **NHIỆM VỤ:** Tạo nội dung chi tiết cho MỘT hoạt động giảng giải lý thuyết.
    **MÔ TẢ YÊU CẦU:** "{task_to_run.get('task_description')}"
    **CHỦ ĐỀ:** {state.get('analyzed_objective', {}).get('topic')}
    **YÊU CẦU:** Nội dung phải cụ thể, có thể là các gạch đầu dòng kiến thức chính hoặc các câu hỏi gợi mở.
    **YÊU CẦU ĐỊNH DẠNG:** Trả về JSON theo schema `LearningActivity`.
    """
    structured_llm = llm.with_structured_output(LearningActivity, method="json_mode")
    try:
        activity_result = await structured_llm.ainvoke(prompt)
        activity_result.duration_minutes = task_to_run.get('estimated_duration', 10)
        print_result(activity_result.dict(), "Hoạt động lý thuyết đã thiết kế")
        return {"current_content_to_validate": activity_result.dict()}
    except Exception as e:
        print_warning(f"Lỗi tại Theory Synthesizer: {e}. Trả về nội dung mặc định.")
        return {"current_content_to_validate": {"error": str(e)}}

async def activity_designer_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Activity Designer` đang thiết kế hoạt động vận dụng...")
    task_to_run = next((t for t in state.get('task_list', []) if t['task_id'] == state.get('current_task_id')), None)
    if not task_to_run: return {"current_content_to_validate": {"error": "Không tìm thấy task."}}
    
    prompt = f"""{SYSTEM_PERSONA_PROMPT}
    **NHIỆM VỤ:** Thiết kế MỘT hoạt động vận dụng, thực hành cho học sinh.
    **MÔ TẢ YÊU CẦU:** "{task_to_run.get('task_description')}"
    **CHỦ ĐỀ:** {state.get('analyzed_objective', {}).get('topic')}
    **YÊU CẦU:** Phải tạo ra các bài tập, câu hỏi thảo luận cụ thể.
    **YÊU CẦU ĐỊNH DẠNG:** Trả về JSON theo schema `LearningActivity`.
    """
    structured_llm = llm.with_structured_output(LearningActivity, method="json_mode")
    try:
        activity_result = await structured_llm.ainvoke(prompt)
        activity_result.duration_minutes = task_to_run.get('estimated_duration', 15)
        print_result(activity_result.dict(), "Hoạt động vận dụng đã thiết kế")
        return {"current_content_to_validate": activity_result.dict()}
    except Exception as e:
        print_warning(f"Lỗi tại Activity Designer: {e}. Trả về nội dung mặc định.")
        return {"current_content_to_validate": {"error": str(e)}}

async def assessment_creator_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Assessment Creator` đang tạo bài toán đánh giá...")
    task_to_run = next((t for t in state.get('task_list', []) if t['task_id'] == state.get('current_task_id')), None)
    if not task_to_run: return {"current_content_to_validate": {"error": "Không tìm thấy task."}}
    
    prompt = f"""{SYSTEM_PERSONA_PROMPT}
    **NHIỆM VỤ:** Tạo MỘT bài toán để đánh giá học sinh.
    **MÔ TẢ YÊU CẦU:** "{task_to_run.get('task_description')}"
    **CHỦ ĐỀ:** {state.get('analyzed_objective', {}).get('topic')}
    **YÊU CẦU:** Bài toán phải có đề bài rõ ràng, đáp án và lời giải chi tiết.
    **YÊU CẦU ĐỊNH DẠNG:** Trả về JSON theo schema `AssessmentItem`.
    """
    structured_llm = llm.with_structured_output(AssessmentItem, method="json_mode")
    try:
        assessment_result = await structured_llm.ainvoke(prompt)
        print_result(assessment_result.dict(), "Câu hỏi đánh giá đã tạo")
        # Thêm duration vào để nhất quán, dù model không yêu cầu
        content_dict = assessment_result.dict()
        content_dict['duration_minutes'] = task_to_run.get('estimated_duration', 10)
        return {"current_content_to_validate": content_dict}
    except Exception as e:
        print_warning(f"Lỗi tại Assessment Creator: {e}. Trả về nội dung mặc định.")
        return {"current_content_to_validate": {"error": str(e)}}

In [20]:
# ==============================================================================
# CELL 6: AGENTS - GIAI ĐOẠN GIÁM SÁT & TỔNG HỢP
# ==============================================================================

async def content_validator_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Contextual Critic` đang phản biện nội dung...")
    content = state.get('current_content_to_validate')
    if not content or content.get("error"):
        print_warning("Bỏ qua phản biện do lỗi ở bước tạo nội dung.")
        return {"validation_feedback": "Lỗi tạo nội dung, không thể phản biện."}
    
    prompt = f"""BẠN LÀ MỘT GIÁO VIÊN LỚP {state.get('analyzed_objective', {}).get('grade_level', '9')} CỰC KỲ KINH NGHIỆM VÀ CẨN THẬN, VỚI CHUYÊN MÔN SÂU VỀ **{state.get('domain', 'Toán học')}**.
    **BỐI CẢNH:** Buổi học hôm nay có chủ đề chính là **"{state.get('analyzed_objective', {}).get('topic', '')}"**.
    **NHIỆM VỤ:** Hãy xem xét nội dung được tạo ra cho một hoạt động trong giáo án.
    **NỘI DUNG CẦN XEM XÉT:**
    ```json
    {json.dumps(content, ensure_ascii=False, indent=2)}
    ```
    **TIÊU CHÍ PHẢN BIỆN:**
    1.  **Tính liên quan:** Nội dung có liên quan trực tiếp đến chủ đề chính của bài học không?
    2.  **Tính chính xác chuyên môn:** Nội dung có sai sót nào về kiến thức, công thức, logic không?
    **YÊU CẦU ĐỊNH DẠNG:** Trả về JSON với 2 key `is_valid` (boolean) và `feedback` (string).
    """
    structured_llm = llm.with_structured_output(ValidationResult, method="json_mode")
    try:
        result = await structured_llm.ainvoke(prompt)
        if not result.is_valid:
            print_warning(f"PHẢN BIỆN: {result.feedback}")
            return {"validation_feedback": result.feedback}
        else:
            print_step("✔️ CHUYÊN GIA PHẢN BIỆN: Nội dung hợp lệ!")
            return {"validation_feedback": None}
    except Exception as e:
        print_warning(f"Lỗi tại Content Validator: {e}. Mặc định cho qua.")
        return {"validation_feedback": None}

def commit_validated_content_node(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Commiter`: Đang ghi nhận nội dung đã được xác thực...")
    agent_ran = state.get("next_agent")
    content = state.get("current_content_to_validate")
    current_task_id = state.get("current_task_id")
    
    if content and agent_ran and current_task_id is not None:
        content['task_id'] = current_task_id
        key_to_update = "assessments" if agent_ran == "assessment_creator" else "activities"
        updated_outputs = update_agent_outputs(state, key_to_update, content)
        updated_outputs["current_content_to_validate"] = None
        return updated_outputs
    return {}

async def plan_compiler_and_critic_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Plan Compiler & Critic` đang tổng hợp và đánh giá...")
    outputs = state.get('agent_outputs', {})
    objective = state.get('analyzed_objective', {})
    activities = outputs.get('activities', [])
    assessments = outputs.get('assessments', [])
    all_content_items = activities + assessments
    total_duration = sum(item.get('duration_minutes', 0) for item in all_content_items)
    allowed_duration = objective.get('constraints', {}).get('duration_minutes', 90)
    is_duration_valid = (allowed_duration * 0.85) <= total_duration <= (allowed_duration + 10)

    if not all_content_items or len(all_content_items) < 3 or not is_duration_valid:
        reflection = f"Kế hoạch thất bại. Tổng thời gian thực tế ({total_duration} phút) nằm ngoài khoảng cho phép. Hoặc số lượng hoạt động ({len(all_content_items)}) quá ít. Hãy lập kế hoạch lại."
        print_warning(f"CRITIC: {reflection}")
        return {"reflection_notes": reflection, "final_lesson_plan": None}

    print_step("CRITIC: Kế hoạch hợp lệ! Bắt đầu tổng hợp giáo án chi tiết...")
    all_content_sorted = sorted(all_content_items, key=lambda x: x.get('task_id', float('inf')))
    content_md_parts = []
    
    for i, content in enumerate(all_content_sorted):
        title = f"### Hoạt động {i+1}: {content.get('activity_name', 'N/A')} ({content.get('duration_minutes', 0)} phút)"
        if 'question' in content:
            details = (f"*   **Loại hình:** {content.get('question_type', 'Đánh giá')}\n"
                       f"*   **Đề bài:**\n{content.get('question', 'N/A')}\n\n"
                       f"*   **Đáp án:** {content.get('answer', 'N/A')}\n"
                       f"*   **Hướng dẫn giải chi tiết:**\n{format_solution_guide(content.get('solution_guide'))}\n")
        else:
            details = (f"*   **Loại hình:** {content.get('activity_type', 'N/A')}\n"
                       f"*   **Mô tả/Nhiệm vụ:**\n{content.get('description', 'N/A')}\n\n"
                       f"*   **Gợi ý đáp án/Hướng dẫn giải:**\n{format_solution_guide(content.get('solution_guide'))}\n")
        content_md_parts.append(f"{title}\n\n{details}")
    
    content_md = "\n---\n\n".join(content_md_parts)
    final_plan_str = f"""# GIÁO ÁN BÀI DẠY: {objective.get('topic', 'N/A')}
---
## I. THÔNG TIN CHUNG
- **Môn học:** {state.get('domain', 'N/A')}
- **Lớp:** {objective.get('grade_level', 'N/A')}
- **Thời lượng dự kiến:** {total_duration} phút / {allowed_duration} phút
- **Phương pháp sư phạm chủ đạo:** {state.get('pedagogy_strategy', {}).get('chosen_pedagogy', 'Chưa xác định')}
---
## II. MỤC TIÊU BÀI HỌC
- Học sinh có thể **{objective.get('action_verb', 'vận dụng')}** kiến thức về {objective.get('topic', 'N/A')} để giải quyết các bài toán liên quan.
---
## III. CHUẨN BỊ
- **Giáo viên:** Bảng phụ, phấn màu, phiếu học tập.
- **Học sinh:** Sách giáo khoa, vở ghi, dụng cụ học tập.
- **Nguồn tài liệu tham khảo:** {state.get('agent_outputs', {}).get('resources', [{}])[0].get('source', 'N/A')}
---
## IV. TIẾN TRÌNH BÀI DẠY
{content_md}
---
## V. TÓM TẮT KIẾN THỨC CỐT LÕI
{state.get('agent_outputs', {}).get('resources', [{}])[0].get('summary', 'Chưa có tóm tắt.')}
"""
    return {"final_lesson_plan": final_plan_str, "reflection_notes": None}

In [21]:
# ==============================================================================
# CELL 7: LẮP RÁP GRAPH & CHẠY THỬ NGHIỆM
# ==============================================================================

# --- CÁC NODE TIỆN ÍCH VÀ ĐIỀU PHỐI ---
def mark_task_complete(state: "TeacherStateV4") -> Dict[str, Any]:
    task_list = state.get("task_list", [])
    current_task_id = state.get("current_task_id")
    if not task_list or current_task_id is None: return {}
    new_task_list = [t.copy() for t in task_list]
    for task in new_task_list:
        if task.get("task_id") == current_task_id:
            task["status"] = "completed"
            break
    return {"task_list": new_task_list, "current_task_id": None}
    
def task_router_node(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Router`: Đang kiểm tra nhiệm vụ...")
    task_list = state.get("task_list", [])
    next_task = next((task for task in task_list if task.get("status") == "pending"), None)
    if next_task:
        print_step(f"Nhiệm vụ tiếp theo: '{next_task['task_name']}' (ID: {next_task['task_id']})")
        return {"current_task_id": next_task['task_id']}
    else:
        print_step("Hết nhiệm vụ, chuyển sang node Compiler & Critic.")
        return {"current_task_id": None}

async def task_dispatcher_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Task Dispatcher` Bắt đầu...")
    task_to_run = next((t for t in state.get('task_list', []) if t['task_id'] == state.get('current_task_id')), None)
    if not task_to_run:
        print_warning("Dispatcher không tìm thấy task, sẽ bỏ qua bước này.")
        return {"next_agent": "mark_task_complete"}

    task_description = task_to_run.get('task_description', '')
    prompt = f"{SYSTEM_PERSONA_PROMPT}\n**Nhiệm vụ:** Phân loại nhiệm vụ sau vào MỘT trong ba agent sau: 'activity_designer', 'theory_synthesizer', 'assessment_creator'.\n\n**QUY TẮC PHÂN LOẠI:**\n- Giảng giải, giới thiệu, ôn tập, tổng kết -> 'theory_synthesizer'.\n- Luyện tập, thực hành, thảo luận -> 'activity_designer'.\n- Kiểm tra, đánh giá, vận dụng cao -> 'assessment_creator'.\n\n**NHIỆM VỤ CẦN PHÂN LOẠI:** \"{task_description}\"\n\n**YÊU CẦU JSON:** Trả về JSON với key duy nhất là `agent_category`."
    structured_llm = llm.with_structured_output(TaskClassification)
    try:
        classification_result = await structured_llm.ainvoke(prompt)
        next_agent = classification_result.agent_category
    except Exception as e:
        print_warning(f"Lỗi tại Dispatcher: {e}. Giao nhiệm vụ cho 'activity_designer'.")
        next_agent = "activity_designer"
    print_step(f"--- 🚚 `Dispatcher`: Giao nhiệm vụ cho `{next_agent}`.")
    return {"next_agent": next_agent}

# --- CÁC HÀM ĐIỀU HƯỚNG ---
def route_after_validation(state: "TeacherStateV4") -> Literal["commit", "retry"]:
    if state.get("validation_feedback") is None: return "commit"
    else: return "retry"

def route_after_router(state: "TeacherStateV4") -> Literal["continue_executing", "compile_and_critique"]:
    if state.get("current_task_id") is None: return "compile_and_critique"
    else: return "continue_executing"

def route_after_compilation(state: "TeacherStateV4") -> Literal["finish", "replan"]:
    if state.get("reflection_notes"): return "replan"
    else: return "finish"

# --- XÂY DỰNG GRAPH ---
workflow = StateGraph(TeacherStateV4)

# Giai đoạn 1: Phân tích & Lập chiến lược
workflow.add_node("initializer", initialize_state_node)
workflow.add_node("objective_interpreter", objective_interpreter_agent)
workflow.add_node("pedagogy_strategist", pedagogy_strategist_agent)

# Giai đoạn 2: Thu thập & Lập kế hoạch
workflow.add_node("query_expansion", query_expansion_agent)
workflow.add_node("resource_scout", resource_scout_agent_v2)
workflow.add_node("plan_delegator", plan_delegator_agent)

# Giai đoạn 3: Vòng lặp Thực thi & Kiểm duyệt
workflow.add_node("task_router", task_router_node)
workflow.add_node("task_dispatcher", task_dispatcher_agent)
workflow.add_node("theory_synthesizer", theory_synthesizer_agent)
workflow.add_node("activity_designer", activity_designer_agent)
workflow.add_node("assessment_creator", assessment_creator_agent)
workflow.add_node("content_validator", content_validator_agent)
workflow.add_node("commit_validated_content", commit_validated_content_node)
workflow.add_node("mark_task_complete", mark_task_complete)

# Giai đoạn 4: Tổng hợp & Phê bình
workflow.add_node("plan_compiler_and_critic", plan_compiler_and_critic_agent)

# Kết nối dây chuyền
workflow.set_entry_point("initializer")
workflow.add_edge("initializer", "objective_interpreter")
workflow.add_edge("objective_interpreter", "pedagogy_strategist")
workflow.add_edge("pedagogy_strategist", "query_expansion")
workflow.add_edge("query_expansion", "resource_scout")
workflow.add_edge("resource_scout", "plan_delegator")
workflow.add_edge("plan_delegator", "task_router")

# Vòng lặp chính
workflow.add_conditional_edges("task_router", route_after_router, {"continue_executing": "task_dispatcher", "compile_and_critique": "plan_compiler_and_critic"})
workflow.add_conditional_edges("task_dispatcher", lambda state: state.get("next_agent", "activity_designer"), {"activity_designer": "activity_designer", "assessment_creator": "assessment_creator", "theory_synthesizer": "theory_synthesizer"})

# Vòng lặp kiểm duyệt vi mô
workflow.add_edge("theory_synthesizer", "content_validator")
workflow.add_edge("activity_designer", "content_validator")
workflow.add_edge("assessment_creator", "content_validator")
workflow.add_conditional_edges("content_validator", route_after_validation, {"commit": "commit_validated_content", "retry": "task_dispatcher"})
workflow.add_edge("commit_validated_content", "mark_task_complete")
workflow.add_edge("mark_task_complete", "task_router")

# Vòng lặp phê bình vĩ mô
workflow.add_conditional_edges("plan_compiler_and_critic", route_after_compilation, {"replan": "plan_delegator", "finish": END})

app = workflow.compile()
print("✅ Graph phiên bản 'Báo cáo' đã được biên dịch thành công.")

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Lỗi vẽ biểu đồ: {e}")

# --- CHẠY THỬ NGHIỆM ---
async def run_final_version(user_request: str):
    if 'llm' not in globals() or llm is None or 'vector_store' not in globals() or vector_store is None or 'app' not in globals():
        print_warning("Một trong các thành phần cốt lõi (LLM, Vector Store, App) chưa được khởi tạo. Vui lòng chạy lại các cell trên.")
        return

    initial_state = {"messages": [HumanMessage(content=user_request)]}
    console.print(f"\n[bold magenta]🚀 BẮT ĐẦU QUY TRÌNH PHIÊN BẢN 'BÁO CÁO' VỚI YÊU CẦU:[/bold magenta]\n> {user_request}")
    config = {"recursion_limit": 150} 
    
    final_state_result = None
    try:
        async for event in app.astream(initial_state, config=config):
            for node_name, node_output in event.items():
                console.print(f"\n[bold yellow]------- Hoàn thành bước: {node_name} -------[/bold yellow]")
                final_state_result = node_output

        console.print("\n[bold magenta]🏁 KẾT THÚC QUY TRÌNH.[/bold magenta]")
        
        if final_state_result and final_state_result.get("final_lesson_plan"):
            console.print("\n[bold green]📝 GIÁO ÁN HOÀN CHỈNH[/bold green]")
            console.print(Markdown(final_state_result["final_lesson_plan"]))
        else:
            print_warning("Không tạo được giáo án cuối cùng.")
            pprint(final_state_result)
            
    except Exception as e:
        print_warning(f"Đã xảy ra lỗi nghiêm trọng: {e}")
        import traceback
        traceback.print_exc()

report_request = "Soạn giúp tôi giáo án bài 'Đường tròn ngoại tiếp và đường tròn nội tiếp' cho học sinh lớp 9 trong 90 phút. Đây là một lớp học khá yếu, các em thường mất tập trung và cần các hoạt động có tính tương tác cao."
VERBOSE_MODE = True 

await run_final_version(report_request)

NameError: name 'initialize_state_node' is not defined